In [3]:
import xlwt;
from datetime import datetime;
from xlrd import open_workbook;
from xlwt import Workbook;
from xlutils.copy import copy
from pathlib import Path

'''style0 = xlwt.easyxf('font: name Times New Roman, color-index red, bold on',
    num_format_str='#,##0.00')
style1 = xlwt.easyxf(num_format_str='D-MMM-YY')

wb = xlwt.Workbook()
ws = wb.add_sheet('A Test Sheet')

ws.write(0, 0, 1234.56, style0)
ws.write(1, 0, datetime.now(), style1)
ws.write(2, 0, 1)
ws.write(2, 1, 1)
ws.write(2, 2, xlwt.Formula("A3+B3"))

wb.save('example.xls')
'''
def output(filename, sheet,num, name, present):
    my_file = Path('Report/'+filename+str(datetime.now().date())+'.xls');
    if my_file.is_file():
        rb = open_workbook('Report/'+filename+str(datetime.now().date())+'.xls');
        book = copy(rb);
        sh = book.get_sheet(0)
        # file exists
    else:
        book = xlwt.Workbook()
        sh = book.add_sheet(sheet)
    style0 = xlwt.easyxf('font: name Times New Roman, color-index red, bold on',
                         num_format_str='#,##0.00')
    style1 = xlwt.easyxf(num_format_str='D-MMM-YY')

    #variables = [x, y, z]
    #x_desc = 'Display'
    #y_desc = 'Dominance'
    #z_desc = 'Test'
    #desc = [x_desc, y_desc, z_desc]
    sh.write(0,0,datetime.now().date(),style1);


    col1_name = 'Name'
    col2_name = 'Present'


    sh.write(1,0,col1_name,style0);
    sh.write(1, 1, col2_name,style0);
    name_str=str(name)
    sh.write(num+1,0,name);
    sh.write(num+1, 1, present);
    #You may need to group the variables together
    #for n, (v_desc, v) in enumerate(zip(desc, variables)):
    fullname=filename+str(datetime.now().date())+'.xls';
    book.save('Report/'+fullname)
    return fullname;

excel sheet updated


In [ ]:
import cv2
import os
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the pre-trained face classifier
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def crop_faces(image_path):
    # Load the image
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Detect faces in the image
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)
    
    # Crop and save each face
    cropped_faces = []
    bounding_boxes = []
    for i, (x, y, w, h) in enumerate(faces):
        face = img[y:y+h, x:x+w]
        face = cv2.resize(face, (200, 200))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        cropped_faces.append(face)
        bounding_boxes.append((x, y, w, h))
    
    return cropped_faces, bounding_boxes

def predict_and_label(cropped_faces):
    # Make predictions for each cropped face
    predicted_labels = []
    for face in cropped_faces:
        # If the image is grayscale, convert to RGB
        if len(face.shape) == 2:
            face = cv2.cvtColor(face, cv2.COLOR_GRAY2RGB)
        
        # Resize the image to match the input size expected by the model
        face = cv2.resize(face, (200, 200))
        
        img_array = np.expand_dims(face, axis=0)
        img_array = img_array / 255.0
        def get_actual_label(label_index):
            return label_encoder.inverse_transform([label_index])[0]
        predictions = model.predict(img_array)
        predicted_label = [np.argmax(predictions[0])]
        predicted_labels.append(get_actual_label(predicted_label[0]))

    return predicted_labels

def display_results(image_path, bounding_boxes, predicted_labels):
    # Display the original image
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Display bounding boxes and labels
    for (x, y, w, h), label in zip(bounding_boxes, predicted_labels):
        color = (0, 255, 0)  # Green color for bounding box
        cv2.rectangle(img_rgb, (x, y), (x + w, y + h), color, 2)
        cv2.putText(img_rgb, str(label), (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
        
        # Convert label to string before using it in output function
        

    # Display the image with bounding boxes and labels
    plt.imshow(img_rgb)
    plt.title("Face Detection and Labeling")
    plt.axis('off')
    plt.show()

def mark_attendance(predicted_labels, attendance_dict):
   
    for label in predicted_labels:
        if label not in attendance_dict:
            attendance_dict[label] = 'Present'
        else:
            attendance_dict[label] = 'Absent'
    output('attendance','class1',attendance_dict)        

# Example usage
image_path = "data/image1.jpg"
attendance_dict = {}  # Dictionary to store attendance information
cropped_faces, bounding_boxes = crop_faces(image_path)
predicted_labels = predict_and_label(cropped_faces)
mark_attendance(predicted_labels, attendance_dict)

# Display the results
display_results(image_path, bounding_boxes, predicted_labels)

# Print the attendance information
print("Attendance:")
for label, status in attendance_dict.items():
    print(f"{label}: {status}")
